<a id="toc"></a>
# Table of Contents
1. [Import libraries](#import_libraries)
1. [Configure hyper-parameters](#configure_hyper_parameters)
1. [Define helper-functions](#define_helper_functions)
1. [Resize images and corresponding bboxes](#resize_images_and_corresponding_bboxes)
1. [Save and compress the results](#save_and_compress_the_result)

<a id="import_libraries"></a>
# Import libraries
[Bach to Table of Contents](#toc)

In [39]:
#!pip install --upgrade albumentations
#!pip install --upgrade opencv
#!pip uninstall opencv-python
#!pip install opencv-python
#!pip install opencv-contrib-python
#!pip install albumentations==0.4.6
import pathlib
from pathlib import Path
import json

import numpy as np
import pandas as pd
import cv2
import albumentations as A
from tqdm import tqdm
import os

In [40]:
# from google.colab import drive
# drive.mount('/content/drive')

<a id="configure_hyper_parameters"></a>
# Configure hyper-parameters
[Bach to Table of Contents](#toc)

In [41]:
ROOT = Path('I:\\My Drive\\UAS_Beans\\Beans_StandCount\\2022\\SVREC\\ByStep')

try:
    os.mkdir(ROOT / 'f._Resize_img_annot/Plots_img')

except:
    pass

In [42]:

IMG_DIR = ROOT / 'c._ClipPlots\\Plots_img'

ANNOT_DIR = ROOT / "e._ReadingAnnot"

## Defining the image size to be saved
IMG_SIZE_W = 3872
IMG_SIZE_H = 640

<a id="define_helper_functions"></a>
# Define helper-functions
[Bach to Table of Contents](#toc)

In [43]:
def load_dataframe(csv_path: pathlib.PosixPath, image_dir: pathlib.PosixPath) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    
    # Merge all bboxes of each corresponding image
    # Format: [[x1 y1 w1 h1], [x2 y2 w2 h2], [x3 y3 w3 h3], ...]
    df.bbox = df.bbox.apply(lambda x: ' '.join(np.array(json.loads(x), dtype=str)))
    df.bbox = df.groupby(['image_id']).bbox.transform(lambda x: '|'.join(x))
    df.drop_duplicates(inplace=True, ignore_index=True)
    df.bbox = df.bbox.apply(lambda x: np.array([item.split(' ') for item in x.split('|')], dtype=np.float32).tolist())
    
    # Create a path to each image
    df['image_path'] = df.image_id.apply(lambda x: str(image_dir / (x + '.png')))
    
    return df

def load_image(image_path: str) -> np.array:
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    return image

def fix_out_of_range(bbox: list, max_size: int = 3894) -> list:
    bbox[2] = min(bbox[2], max_size - bbox[0])
    bbox[3] = min(bbox[3], max_size - bbox[1])

    return bbox

In [44]:
df = load_dataframe(ANNOT_DIR / 'Annot_beans_22_SVREC_7m__2.csv', IMG_DIR)

In [45]:
df
#df.to_csv(ROOT / ANNOT_DIR / 'Annot_beans_22_SVREC_7m__2_test.csv', index=False)


,image_id,width,height,source,bbox,image_path
0,2202_3008_7m,3894,671,bean,"[[548.0, 142.0, 30.0, 33.0], [583.0, 131.0, 28...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
1,2202_3007_7m,3894,670,bean,"[[3602.0, 119.0, 41.0, 30.0], [3306.0, 123.0, ...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
2,2202_3006_7m,3893,671,bean,"[[104.0, 89.0, 55.0, 29.0], [372.0, 89.0, 27.0...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
3,2202_3001_7m,3894,671,bean,"[[167.0, 140.0, 27.0, 28.0], [270.0, 134.0, 32...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
4,2202_3002_7m,3894,670,bean,"[[9.0, 80.0, 32.0, 33.0], [112.0, 75.0, 25.0, ...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
...,...,...,...,...,...,...
135,2202_2001_7m,3894,670,bean,"[[115.0, 103.0, 25.0, 58.0], [199.0, 111.0, 38...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
136,2202_2002_7m,3894,670,bean,"[[85.0, 113.0, 32.0, 40.0], [336.0, 110.0, 36....",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
137,2202_2003_7m,3894,670,bean,"[[226.0, 126.0, 53.0, 25.0], [298.0, 115.0, 29...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...
138,2202_2004_7m,3894,671,bean,"[[475.0, 86.0, 34.0, 42.0], [516.0, 85.0, 30.0...",I:\My Drive\UAS_Beans\Beans_StandCount\2022\SV...


<a id="resize_images_and_corresponding_bboxes"></a>
# Resize images and corresponding bboxes
[Bach to Table of Contents](#toc)

In [46]:
transform = A.Compose(
    [
        A.Resize(height=IMG_SIZE_H, width=IMG_SIZE_W, p=1),
    ], 
    p=1.0, 
    bbox_params=A.BboxParams(
        format='coco',
        min_area=0, 
        min_visibility=0,
        label_fields=['labels']
    )
)

list_of_image_ids = []
list_of_bboxes = []
list_of_sources = []

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    image = load_image(row.image_path)
    # print(row.image_path)
    bboxes = row.bbox

    # Fix "out-of-range" bboxes
    bboxes = [fix_out_of_range(bbox) for bbox in bboxes]
    
    result = transform(image=image, bboxes=bboxes, labels=np.ones(len(bboxes)))
    new_image = result['image']
    new_bboxes = np.array(result['bboxes']).tolist()
    
    # Save new image
    cv2.imwrite(str(ROOT / 'f._Resize_img_annot/Plots_img' / (row.image_id + '.png')), new_image)

    for new_bbox in new_bboxes:
        list_of_image_ids.append(row.image_id)
        list_of_bboxes.append(new_bbox)
        list_of_sources.append(row.source)

100%|██████████| 140/140 [01:12<00:00,  1.94it/s]


In [47]:
new_data_dict = {
    'image_id': list_of_image_ids,
    'width': [IMG_SIZE_W] * len(list_of_image_ids),
    'height': [IMG_SIZE_H] * len(list_of_image_ids),
    'bbox': list_of_bboxes,
    'source': list_of_sources
}

In [48]:
new_df = pd.DataFrame(new_data_dict)

In [49]:
new_df 


,image_id,width,height,bbox,source
0,2202_3008_7m,3872,640,"[544.9039548022599, 135.4396423248882, 29.8305...",bean
1,2202_3008_7m,3872,640,"[579.7062146892656, 124.9478390461997, 27.8418...",bean
2,2202_3008_7m,3872,640,"[609.5367231638418, 141.1624441132638, 11.9322...",bean
3,2202_3008_7m,3872,640,"[648.316384180791, 123.99403874813711, 30.8248...",bean
4,2202_3008_7m,3872,640,"[688.090395480226, 146.88524590163934, 29.8305...",bean
...,...,...,...,...,...
17254,2202_2005_7m,3872,640,"[3362.892655367232, 172.63785394932935, 33.807...",bean
17255,2202_2005_7m,3872,640,"[3503.0960451977403, 516.0059612518628, 31.819...",bean
17256,2202_2005_7m,3872,640,"[3344.0, 513.1445603576751, 27.84180790960454,...",bean
17257,2202_2005_7m,3872,640,"[3186.892655367232, 509.32935916542476, 34.802...",bean


<a id="save_and_compress_the_result"></a>
# Save and compress the results
[Bach to Table of Contents](#toc)

In [50]:
new_df.to_csv(ROOT / 'f._Resize_img_annot/Annot_beans_22_SVREC_7m__3.csv', index=False)